In [1]:
import duckdb
import os

# Get Wasabi credentials from environment
wasabi_endpoint = os.getenv('WASABI_ENDPOINT', 's3.us-east-2.wasabisys.com')
wasabi_access_key = os.getenv('WASABI_ACCESS_KEY')
wasabi_secret_key = os.getenv('WASABI_SECRET_KEY')
bucket_name = os.getenv('WASABI_BUCKET_NAME')

# Create DuckDB connection
con = duckdb.connect()

# Configure S3 settings
con.execute(f"""
    SET s3_endpoint='{wasabi_endpoint}';
    SET s3_access_key_id='{wasabi_access_key}';
    SET s3_secret_access_key='{wasabi_secret_key}';
    SET s3_url_style='path';
    SET preserve_insertion_order = false;
    SET enable_progress_bar = true;
""")

print("✓ DuckDB configured with S3 credentials")
print(f"Endpoint: {wasabi_endpoint}")
print(f"Access Key ID: {wasabi_access_key}")
print(f"Secret Access Key: {wasabi_secret_key}")
print('Bucket Name: ', bucket_name, '\n\n')


✓ DuckDB configured with S3 credentials
Endpoint: s3.us-east-2.wasabisys.com
Access Key ID: 60DLEZ81XUCI9ZNG8M3F
Secret Access Key: PuOySWLzPCtuts4IfXzWzdHRBefvGKgD0laHOb4B
Bucket Name:  dfscrunch-data-lake 




In [21]:
lines_path = f"s3://dfscrunch-data-lake/dds/NHL/moneypuck/lines/data.parquet"
players_path = f's3://dfscrunch-data-lake/dds/NHL/players/*/2025-1*-*/data.parquet'
df_1 = con.execute(
  f"""
  with players as (
  select * from read_parquet('{players_path}', filename=true)
  ),
  lines as (
  select * from read_parquet('{lines_path}', filename=true)
  ),
  players_teams as (
  select distinct current_team from players),
  lines_teams as (
  select distinct team from lines)
  select * from players
"""
).df()

In [14]:

df_2 = con.execute(
    f"""
  with players as (
  select * from read_parquet('{players_path}', filename=true)
  ),
  lines as (
  select * from read_parquet('{lines_path}', filename=true)
  ),
  players_teams as (
  select distinct current_team from players),
  lines_teams as (
  select distinct team from lines)
  select * from lines_teams
"""
).df()


In [20]:
df_1.head()

,player_key,player_id,first_name,last_name,full_name,salary,position,roster_position,current_team,current_team_id,event_id,event_team_id,home_visitor,fav_dog,proj_points,ownership,actual_points,stat_details,made_cut,filename
0,263498:0,263498,Uvis,Balinskis,Uvis Balinskis,2500,D,D,FLA,2047,8397070,83970702,Home,Favorite,0.00,0.00,0.0,,0,s3://dfscrunch-data-lake/dds/NHL/players/dk_cl...
1,204727:0,204727,Frank,Nazar,Frank Nazar,4200,C,C,CHI,2205,8397070,83970701,Visitor,Dog,7.43,11.13,16.3,,0,s3://dfscrunch-data-lake/dds/NHL/players/dk_cl...
2,164964:0,164964,Ville,Koivunen,Ville Koivunen,4900,LW,W,PIT,1927,8397060,83970601,Visitor,Dog,6.41,8.97,1.3,,0,s3://dfscrunch-data-lake/dds/NHL/players/dk_cl...
3,162963:0,162963,Daniil,Tarasov,Daniil Tarasov,8200,G,G,FLA,2047,8397070,83970702,Home,Favorite,0.00,0.00,0.0,,0,s3://dfscrunch-data-lake/dds/NHL/players/dk_cl...
4,158561:0,158561,Brett,Berard,Brett Berard,2800,LW,W,,2059,8397060,83970602,Home,Favorite,5.55,0.36,0.0,,0,s3://dfscrunch-data-lake/dds/NHL/players/dk_cl...


In [16]:
set(df_1.current_team) ^ set(df_2.team)

{'',
 'CBJ',
 'CLB',
 'LA',
 'LAK',
 'NAS',
 'NSH',
 'SJ',
 'SJS',
 'TB',
 'TBL',
 'WAS',
 'WSH'}

In [18]:
set(df_1.current_team)

{'',
 'ANA',
 'BOS',
 'BUF',
 'CAR',
 'CGY',
 'CHI',
 'CLB',
 'COL',
 'DAL',
 'DET',
 'EDM',
 'FLA',
 'LA',
 'MIN',
 'MTL',
 'NAS',
 'NJD',
 'NYI',
 'NYR',
 'OTT',
 'PHI',
 'PIT',
 'SEA',
 'SJ',
 'STL',
 'TB',
 'TOR',
 'UTA',
 'VAN',
 'VGK',
 'WAS',
 'WPG'}